## This script generates a video from a sequence of pictures.

In [1]:
import cv2
import os, sys
from tqdm import tqdm
from videolib import initVideoWriter, imageHiveOverview
sys.path.append("..")
from ForegroundRemoval.PercentileFilter.percentile import percentile_filter_single
import multiprocessing
import pandas as pd
pd.set_option('display.width', 1000)
sys.path.append("..")
from libimage import fetchImagesPaths
from dask.distributed import Client
from dask import compute
# Start a client using all CPUs of this machine
client = Client()
client

Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 12,Total memory: 32.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:58270,Workers: 0
Dashboard: http://127.0.0.1:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B
Comm: tcp://127.0.0.1:58282,Total threads: 3
Dashboard: http://127.0.0.1:58288/status,Memory: 8.00 GiB
Nanny: tcp://127.0.0.1:58273,


## Configuration

In [2]:
# Path to the folder containing the pictures
rootpath = '/Users/cyrilmonette/Library/CloudStorage/SynologyDrive-data/25.07_aSensing_OH/Images/'
rootpath_img_period = 5 # Period between every image in the folder in minutes
hive = "1"

# The following are in CET time zone:
first_dt_str = "250925-115000"
last_dt_str = "251013-160000"

frame_drop = 50 # We keep 1 frame every frame_drop frames. Put one to keep all frames.
fps_video = 10 # Frames per second for the video

remove_bees = True # If True, we remove bees using a percentile filter

## Main code

In [3]:
first_dt = pd.to_datetime(first_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
last_dt = pd.to_datetime(last_dt_str, format="%y%m%d-%H%M%S").tz_localize('CET')
assert first_dt < last_dt, "First date must be before last date"
print("First date: ", first_dt)
print("Last date: ", last_dt)

time_range = pd.date_range(start=first_dt, end=last_dt, freq=f"{rootpath_img_period*frame_drop}min")
time_range = time_range.to_list()
print("Length time range: ", len(time_range))
img_paths = fetchImagesPaths(rootpath, time_range, hive, invalid_recovery_time=120,images_fill_limit=None)
print(img_paths.head())

# Find the first index of img_paths that does not contain Nones in any column
first_valid_index = img_paths.dropna(how='all', inplace=False).index[0]
example_frame = imageHiveOverview([cv2.imread(img_paths.loc[first_valid_index].iloc[j]) for j in range(4)], False,
                                  ["hive" + hive + "_rpi" + str(j+1) + "_" + img_paths.loc[first_valid_index].iloc[j] for j in range(4)])
name = "hive" + hive + "_" + first_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ") + "_" + last_dt.tz_convert('UTC').strftime("%y%m%d-%H%M%SZ")
if remove_bees:
    name += "_beesRemoved"
video = initVideoWriter(dest="outputVideos/", frame=example_frame,name=name, fps=fps_video, grayscale=False)
print("Writing video with dimensions: ", example_frame.shape)

# Iterate over img_paths DataFrame
for i, row in tqdm(img_paths.iterrows(), total=img_paths.shape[0]):
    imgs = []
    names = []
    validity = True 
    if 'valid' in row:
        validity = row['valid']
        row.drop('valid', inplace=True)

    for j in range(len(row)):
        if row.iloc[j] is not None:
            file_path = row.iloc[j]
            names.append(os.path.basename(file_path))  # Get the file name
            if remove_bees:
                _img, _ = percentile_filter_single(file_path,percentile=90, filter_length=20, frame_skip=5, annotate_names=False, verbose=False)
            else:
                _img = cv2.imread(file_path)
            imgs.append(_img)
        else:
            # If the file does not exist, append a black image
            names.append("No image available")
            # Append a black image of the same size as the example frame
            imgs.append(cv2.imread(img_paths.loc[first_valid_index].iloc[j]) * 0)

    if remove_bees:
        imgs = compute(*imgs)
        # Convert the images to rgb from grayscale
        imgs = [cv2.cvtColor(img, cv2.COLOR_GRAY2BGR) if len(img.shape) == 2 else img for img in imgs]
    assembled_img = imageHiveOverview(imgs, img_names=names, dt=i, valid=validity)
    video.write(assembled_img)

# Release the VideoWriter object
video.release()
print("Video written successfully.")

First date:  2025-09-25 11:50:00+02:00
Last date:  2025-10-13 16:00:00+02:00
Length time range:  105
                                                                        h1r1                                               h1r2                                               h1r3                                               h1r4  valid
2025-09-25 11:50:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...   True
2025-09-25 16:00:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...   True
2025-09-25 20:10:00+02:00  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/cyrilmonette/Library/CloudStorage/Synol...  /Users/c

100%|██████████| 105/105 [19:41<00:00, 11.25s/it]

Video written successfully.
